### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)


True

In [15]:
from accounts import Account

In [16]:
account = Account.get("Ed")
account

Account(name='ed', balance=9774.55, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 75.15 each.], portfolio_value_time_series=[('2026-01-02 20:53:39', 10038.55), ('2026-01-02 20:54:17', 9936.55)])

In [17]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9759.519999999999, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 75.15, "timestamp": "2026-01-02 20:53:39", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 5.01, "timestamp": "2026-01-02 21:03:38", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-01-02 20:53:39", 10038.55], ["2026-01-02 20:54:17", 9936.55], ["2026-01-02 21:03:38", 10101.519999999999]], "total_portfolio_value": 10101.519999999999, "total_profit_loss": 101.52000000000044}'

In [18]:
account.report()

'{"name": "ed", "balance": 9759.519999999999, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 75.15, "timestamp": "2026-01-02 20:53:39", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 5.01, "timestamp": "2026-01-02 21:03:38", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-01-02 20:53:39", 10038.55], ["2026-01-02 20:54:17", 9936.55], ["2026-01-02 21:03:38", 10101.519999999999], ["2026-01-02 21:03:39", 9963.519999999999]], "total_portfolio_value": 9963.519999999999, "total_profit_loss": -36.47999999999956}'

In [19]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 75.15,
  'timestamp': '2026-01-02 20:53:39',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 5.01,
  'timestamp': '2026-01-02 21:03:38',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [4]:
# Now let's use our accounts server as an MCP server
from agents.mcp import MCPServerStdio
params = {
    "command": "python",
    "args": ["accounts_server.py"]
}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

In [5]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=

In [6]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4o-mini"

In [12]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="account_manager",
        instructions=instructions, 
        model=model, 
        mcp_servers=[mcp_server]
    )
    with trace("account_manager"):
        result =await Runner.run(agent,request)
    display(Markdown(result.final_output))

Your account balance is **$9,759.52**. 

You currently hold **6 shares of AMZN (Amazon)**.

In [13]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="account_manager",
        instructions=instructions, 
        model=model, 
        mcp_servers=[mcp_server]
    )
    with trace("account_manager"):
        result =await Runner.run(agent,request)
    display(Markdown(result.final_output))

Your current balance is **$9,759.52**. 

In terms of holdings, you have **6 shares of Amazon (AMZN)**.

### Now let's build our own MCP Client

In [18]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)


  + Exception Group Traceback (most recent call last):
  |   File "/opt/miniconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3575, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/3z/0qfr25l12bdgg_sj9z0m2jqc0000gn/T/ipykernel_1170/3381116230.py", line 3, in <module>
  |     mcp_tools = await list_accounts_tools()
  |                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/oluwatobilobasokoya/AI/agents/6_mcp/accounts_client.py", line 11, in list_accounts_tools
  |     server_script = os.path.join(script_dir, "accounts_server.py")
  |                    ^^^^^^^^^^^^^^^^^^^^
  |   File "/opt/miniconda3/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/opt/miniconda3/lib/python3.12/site-packages/mcp/client/stdio/__init__.py", line 183, in stdio_client
  |     anyio.create_task_group() as tg,
  |     ^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/opt/miniconda3

In [ ]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("ed")
print(context)

In [ ]:
from accounts import Account
Account.get("ed").report()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>